<img src="http://python-visualization.github.io/folium/_images/folium_logo.jpg" width="100" align="right">
In this notebook, we'll investigate the interactive mapping capabilities of the Python package called folium. In a nutshell, what folium does is provide a Python wrapper around the Javascript `leaflet.js`, which can create Google Maps-style interactive maps and visualizations for simple data needs, without writing any actual Javascript.

From [folium's website](http://python-visualization.github.io/folium/):
>folium builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the [leaflet.js library](http://leafletjs.com/). Manipulate your data in Python, then visualize it in on a Leaflet map via folium.

## First steps with Folium
The folium interface is surprisingly high-level and easy to use.

In [1]:
import folium

# Canada's centroid is apparently controversial
# https://goo.gl/yDL8UX
canada_centroid = [64.3176, -96.0220]

m = folium.Map(
    location=canada_centroid,  # Centre of point-of-view
    zoom_start=3,              # Zoom level of map
    tiles='Cartodb Positron'   # Style of basemap
)

folium.Marker(
    location=canada_centroid,
    popup="Canada Centroid"
).add_to(m)

m

In [2]:
m.save('canada.html')

## National Pollutant Release Inventory
From the Environment and Climate Change Canada's website:
>The National Pollutant Release Inventory (NPRI) is Canada's legislated, publicly accessible inventory of pollutant releases (to air, water and land), disposals and transfers for recycling. It is a key resource for:

>* identifying pollution prevention priorities;
>* supporting the assessment and risk management of chemicals, and air quality modelling;
>*  helping develop targeted regulations for reducing releases of toxic substances and air pollutants;
>* encouraging actions to reduce the release of pollutants into the environment; and
>* improving public understanding.

The data is available through the [Open Canada Portal](http://open.canada.ca/en/open-data).

In [3]:
import pandas as pd
import fiona
import geopandas as gp
from shapely.geometry import Point

npri_url = r'http://donnees.ec.gc.ca/data/substances/plansreports/national-pollutant-release-inventory-npri-pollutant-release-and-transfer-data-reported-by-facilities/national-pollutant-release-inventory-npri-bulk-data/NPRI-Facility-Geo-Locations-Since1993.csv'
df = pd.read_csv(npri_url)  # I am still convinced this is magic :)

# The dataset has lat/long values, which we can use
# to set Geometry for each of the rows
df['geometry'] = df.apply(lambda z: Point(z['Latitude / Latitude'],
                                          z['Longitude / Longitude']), axis=1)
df = gp.GeoDataFrame(df)
df.crs = fiona.crs.from_epsg(3347)
df.head()

,Year of last filed report / L'année de la déclaration la plus récente,NPRI ID / ID INRP,Company Name / Raison Sociale,Facility Name / Nom de l'installation,Address line 1 / Première ligne d’adresse,Address line 2 / Deuxième ligne d’adresse,City / Ville,Province / Province,Postal Code / Code postal,Physical Land Survey Description / Description de l'arpentage,...,Census Metropolitan Area / Nom de la Région métropolitaine de recensement,Economic Region Unique ID / No unique de la Région économique,Economic Region Name / Nom de la Région économique,Unique Ecozone ID / No unique de l’Écozone,English Ecozone Name / Nom englais de l’Écozone,French Ecozone Name / Nom français de l’Écozone,Unique ID of the Major Drainage Area from the Water Survey of Canada (WSC) / No unique de l’aire de drainage principale des Relevés hydrologiques du Canada (RHC),Major Drainage Area English Name / Nom englais de l’aire de drainage principale,Major Drainage Area French Name / Nom français de l’aire de drainage principale,geometry
0,2015,1,Alberta-Pacific Forest Industries Inc.,Alberta-Pacific Forest Industries Inc.,NaN,NaN,County of Athabasca,AB,T0A0M0,SW-32-068-19-W4,...,NaN,4870.0,Athabasca--Grande Prairie--Peace River,9,Boreal PLain,Plaines boréales,7.0,Great Slave Lake Drainage Area,Aire de drainage du Grand lac des Esclaves,POINT (54.923116 -112.861867)
1,2015,11,Hexion Canada Inc.,Hexion Canada Inc. - Edmonton Facility,12621 156th Street,NaN,Edmonton,AB,T5V1E1,NaN,...,Edmonton,4860.0,Edmonton,10,Prairie,Prairies,5.0,Nelson River Drainage Area,Aire de drainage du fleuve Nelson,POINT (53.5823 -113.5886)
2,2015,15,Baycoat Ltd.,Baycoat Ltd.,244 Lanark Street,NaN,Hamilton,ON,L8N3K7,NaN,...,Hamilton,3550.0,Hamilton--Niagara Peninsula,8,MixedWood Plain,Plaines à forêts mixtes,2.0,St. Lawrence Drainage Area,Aire de drainage du Saint-Laurent,POINT (43.2428 -79.7503)
3,2015,21,Bolton Steel Tube Co. Ltd.,Bolton Plant,455 Piercy Road,NaN,Bolton,ON,L7E5B8,NaN,...,Toronto,3530.0,Toronto,8,MixedWood Plain,Plaines à forêts mixtes,2.0,St. Lawrence Drainage Area,Aire de drainage du Saint-Laurent,POINT (43.8656 -79.7308)
4,2015,28,BASF Canada Inc.,CORNWALL SITE,501 Wallrich Avenue,NaN,Cornwall,ON,K6J2B5,NaN,...,Cornwall,3510.0,Ottawa,8,MixedWood Plain,Plaines à forêts mixtes,2.0,St. Lawrence Drainage Area,Aire de drainage du Saint-Laurent,POINT (45.0189 -74.7551)


Let's concentrate on using only the data from the latest year: 2015.

In [4]:
df = df[df["Year of last filed report / L'année de la déclaration la plus récente"] == 2015]

In [5]:
print('There are',
      df['NPRI ID / ID INRP'].nunique(),
      'unique installations in the 2015 dataset')

There are 8610 unique installations in the 2015 dataset


Next, let's extract the locations and installation names so we can add them to the map as Markers.

In [6]:
locations = [[pt.x, pt.y] for pt in df.geometry.values]

In [7]:
popups = df["Facility Name / Nom de l\'installation"].values

In [22]:
from folium.plugins import FastMarkerCluster
from IPython.display import Image

m = folium.Map(
    location=canada_centroid,
    zoom_start=3,
    tiles='Cartodb Positron'
)

marker_cluster = FastMarkerCluster(data=locations)
marker_cluster.add_to(m)
folium.LayerControl().add_to(m)
m.save('npri.html')

# Save static figure as png
Image(m._repr_png_())


ValueError: No image data found. Expecting filename, url, or data.

## Considerations
For this simple demo, I haven't been able to make the markers have any other "qualities": popups, marker colours by industry type, etc. Total development time so far was only 1h.

[Folium](http://python-visualization.github.io/folium/quickstart.html) still is in active development, and there are many "bells and whistles" that might only be added by coding the underlying Javascript code instead.

On the other hand, its use for simple ["slippy map"](https://en.wikipedia.org/wiki/Tiled_web_map)-type visualisations is bar-none in terms of quality and ease of use.